**本题的核心方法是采用统计分析技术提取图像的颜色、亮度和清晰度特征，以实现对退化类型的精确分类。**
### 颜色偏移（color cast）检测
颜色偏移主要是由于水对不同波长光的吸收差异而引起，通常表现为图像的单一颜色通道（RGB）占主导地位。
通过分别计算RGB三个通道的均值（R为Red通道的均值，G为Green通道的均值，B为Blue通道的均值），分析不同通道的分布不均衡性。
采用公式 Color_Imbalance=max(∣R−G∣,∣R−B∣,∣G−B∣) 计算得到颜色偏移指标，采用cc作为阈值。
当颜色偏移指标超过阈值时，这判断为存在显著的颜色偏移。  

参考文献：  
Ancuti, C. O., Ancuti, C., De Vleeschouwer, C., & Bekaert, P. (2012). Enhancing underwater images and videos by fusion. Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition (CVPR).

### 低光照（low light）检测
低光照是由于水中光的传播损耗导致的，表现为图像亮度较低且对比度不足。
通过将图片转化为灰度图，计算其平均亮度和亮度对比度，采用L1和L2分别作为亮度和对比度的阈值。
若超过阈值，则判断为低光照图像。  

参考文献：  
Iqbal, K., Odetayo, M., & James, A. (2010). Enhancing the low quality images using Unsupervised Color Correction Method. International Conference on Systems, Man and Cybernetics.

### 模糊（blur）检测
模糊通常由前向散射或成像设备性能限制引起，表现为图像细节模糊不清。
采用拉普拉斯算子检测图像的梯度变化，并计算梯度方差。取b为阈值。
若方差超过阈值，则判断为模糊图像。  

参考文献：  
Pertuz, S., Puig, D., & Garcia, M. A. (2013). Analysis of focus measure operators for shape-from-focus. Pattern Recognition.

In [2]:
import cv2
import numpy as np
import os

In [3]:
def classify_images(folder_path, cc, L1, L2, b):
    classifications = {"color_cast":[],
                       "low_light":[],
                       "blur":[]}
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        img = cv2.imread(file_path)
        
        # Color Cast
        B_mean, G_mean, R_mean = cv2.mean(img)[:3]
        color_imbalance = max(abs(R_mean- G_mean), abs(G_mean - B_mean), abs(B_mean - R_mean))
        is_color_cast = color_imbalance > cc
        
        # Low Light
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        is_low_light = np.mean(gray_img) < L1
        is_low_contrast = np.std(gray_img) < L2
        
        # Blur
        Var_Laplacian = cv2.Laplacian(gray_img, cv2.CV_64F).var()
        is_blur = Var_Laplacian < b
        
        if is_color_cast:
            classifications["color_cast"].append(filename)
        if is_low_light or is_low_contrast:
            classifications["low_light"].append(filename)
        if is_blur:
            classifications["blur"].append(filename)
    
    return classifications

In [4]:
images_folder_path = os.path.join(os.path.join(os.getcwd(), "Attachment"), "Attachment 1")
results = classify_images(images_folder_path, 50, 20, 50, 100)

In [5]:
import pandas as pd

df = pd.read_excel('Answer.xls')

for key, value in results.items():
    item_df = pd.DataFrame({
        'Degraded Image Classification': [key] * len(value),
        'image file name': value
    })
    df = pd.concat([df, item_df], ignore_index=True)

df.to_excel('Answer_updated.xlsx', index=False)